# Setup

In [1]:
%pip install spuco

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 24.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.2/126.2 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 6.2 MB/s eta 0:00:00
  Created wheel for grad-cam: filename=grad_cam-1.5.4-py3-none-any.whl size=39588 sha256=265a284123c886d48412eb3014d800a86cfaf54eea30f7f6c2de8eec0a3ffe74
  Stored in directory: /root/.cache/pip/wheels/50/b0/82/1f97b5348c7fe9f0ce0ba18497202cafa5dec4562bd5292680
Successfully built grad-cam


In [2]:
import torch

device = torch.device("cpu")

In [3]:
from spuco.utils import set_seed

set_seed(0)

# Dataset Creation

In [4]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    split="train",
    label_noise=0.001
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:01<00:00, 5.82MB/s]


Extracting /data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 154kB/s]


Extracting /data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:01<00:00, 1.46MB/s]


Extracting /data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 8.00MB/s]


Extracting /data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /data/mnist/MNIST/raw



In [5]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]

valset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="val"
)
valset.initialize()


In [6]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [12]:
from spuco.models import model_factory
from spuco.robust_train import ERM
from torch.optim import SGD
import numpy as np
from spuco.robust_train import CustomSampleERM
from spuco.utils import Trainer
from spuco.group_inference import JTTInference
import random
random.seed(10)

# Train using JTT

In [36]:
#jtt params
jtt_epochs = 2
print('Total trainset size: ' + str(len(trainset)))
shuffled_indices = np.arange(0, len(trainset))
random.shuffle(shuffled_indices)

Total trainset size: 48004


## Group Inference Merge: 2 Groups

In [37]:
def gi_subset(num_partitions, partition_id, trainset, device, shuffled_indices):
  #generate subset index
  partition_len = int(len(trainset)/num_partitions)
  train_index = shuffled_indices[partition_len*partition_id:partition_len*partition_id+partition_len]
  train_index = train_index.tolist()
  print(train_index)
  #train on subset
  infer_model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
  gi_trainer = CustomSampleERM(
      model=infer_model,
      trainset=trainset,
      batch_size=64,
      optimizer=SGD(infer_model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
      num_epochs=2,
      indices=train_index,
      device=device,
      verbose=True
  )
  gi_trainer.train()
  #get labels
  gi_trainer_results = Trainer(
    model=infer_model,
    trainset=trainset,
    batch_size=64,
    optimizer=SGD(infer_model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
  )
  output = gi_trainer_results.get_trainset_outputs()
  max_output = torch.max(output, dim=1)
  pred_labels = max_output.indices
  pred_labels = pred_labels.tolist()
  actual_labels = trainset.labels
  #jtt inference on subset
  jtt = JTTInference([pred_labels[i] for i in train_index], [actual_labels[i] for i in train_index])
  gi = jtt.infer_groups()
  nominal_upsample = len(gi[(0, 0)])/len(gi[(0, 1)])
  print('\nNominal Upsample: ' + str(nominal_upsample))
  print('Majority class size: ' + str(len(gi[(0, 0)])))
  print('Minority class size: ' + str(len(gi[(0, 1)])))
  #return majority, minority group indices
  majority = [train_index[i] for i in gi[(0, 0)]]
  minority = [train_index[i] for i in gi[(0, 1)]]
  return majority, minority

In [28]:
maj1, min1 = gi_subset(2, 0, trainset, device, shuffled_indices)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


[36725, 43002, 34866, 40548, 1729, 24993, 11200, 10557, 20402, 9957, 6444, 31320, 44170, 2533, 37078, 18296, 6907, 721, 32756, 8268, 38465, 32832, 32752, 8330, 10113, 31792, 47116, 9875, 46405, 10816, 18115, 44148, 23618, 2453, 17523, 20259, 36930, 6552, 30035, 2514, 27848, 22058, 10687, 32656, 12147, 10661, 27808, 27006, 14411, 43738, 36808, 679, 42887, 541, 15795, 42566, 47223, 24764, 8875, 2210, 18440, 13805, 12944, 26238, 44693, 39704, 5052, 1245, 46139, 32391, 31934, 157, 28410, 16110, 17863, 13254, 28427, 24141, 46960, 26389, 35728, 9772, 9768, 43015, 22579, 34075, 44230, 46397, 17269, 30268, 34441, 28796, 20117, 28306, 11028, 34728, 7773, 11057, 46098, 20742, 46576, 22026, 23483, 19482, 29054, 36889, 2045, 38201, 28674, 2531, 30729, 3310, 16546, 38284, 34978, 16371, 24583, 40124, 46088, 19769, 9305, 14887, 45888, 17497, 16008, 12980, 34872, 34878, 39707, 4672, 39289, 15273, 12786, 32914, 9029, 14740, 26231, 2483, 22396, 15492, 36697, 15354, 4213, 35324, 17487, 38908, 995, 20696,

Epoch 0:   2%|▏         | 7/376 [00:00<00:17, 20.72batch/s, accuracy=23.4375%, loss=1.6] 

 | Epoch 0 | Loss: 1.6233676671981812 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.595747470855713 | Accuracy: 35.9375%
 | Epoch 0 | Loss: 1.6330136060714722 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.6209677457809448 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.6207256317138672 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.60149085521698 | Accuracy: 26.5625%
 | Epoch 0 | Loss: 1.6115206480026245 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.6081756353378296 | Accuracy: 28.125%
 | Epoch 0 | Loss: 1.6217176914215088 | Accuracy: 6.25%
 | Epoch 0 | Loss: 1.612554669380188 | Accuracy: 20.3125%
 | Epoch 0 | Loss: 1.596153736114502 | Accuracy: 23.4375%


Epoch 0:   5%|▌         | 19/376 [00:00<00:09, 38.41batch/s, accuracy=12.5%, loss=1.6]  

...

 | Epoch 1 | Loss: 0.08560766279697418 | Accuracy: 98.4375%
 | Epoch 1 | Loss: 0.0042868987657129765 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.13439500331878662 | Accuracy: 98.4375%
 | Epoch 1 | Loss: 0.2018832117319107 | Accuracy: 96.875%
 | Epoch 1 | Loss: 0.010447564534842968 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.014980335719883442 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.01766880229115486 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.010658991523087025 | Accuracy: 100.0%


Epoch 1:  99%|█████████▉| 373/376 [00:10<00:00, 37.61batch/s, accuracy=100.0%, loss=0.00169]

 | Epoch 1 | Loss: 0.010969186201691628 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.007489045616239309 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.08819945156574249 | Accuracy: 98.4375%
 | Epoch 1 | Loss: 0.00846655759960413 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.005542383994907141 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.10115118324756622 | Accuracy: 98.4375%
 | Epoch 1 | Loss: 0.005111483857035637 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.0016875239089131355 | Accuracy: 100.0%


Getting model outputs: 100%|██████████| 751/751 [00:05<00:00, 127.82batch/s]


Nominal Upsample: 175.48529411764707
Majority class size: 23866
Minority class size: 136


In [29]:
maj2, min2 = gi_subset(2, 1, trainset, device, shuffled_indices)

[40292, 43686, 15013, 37894, 27982, 45005, 41279, 37626, 46679, 21826, 21012, 22804, 41057, 37255, 2031, 5176, 29424, 25663, 5101, 32990, 42418, 14428, 2875, 31133, 18597, 39740, 38168, 5077, 47289, 4171, 18496, 21476, 16946, 2163, 3938, 35595, 23427, 20042, 42505, 30667, 5531, 9385, 1014, 33226, 4447, 20868, 9546, 16762, 8887, 8922, 27807, 31758, 41520, 27336, 16906, 28983, 42357, 27371, 39495, 3906, 3691, 16202, 20650, 12741, 21509, 17965, 10862, 8376, 23836, 35297, 36124, 18643, 10214, 30693, 28788, 24544, 28676, 41513, 21641, 34952, 27733, 16598, 298, 33868, 7642, 10297, 21741, 33936, 14655, 21658, 23093, 33966, 5606, 6171, 35828, 47494, 43622, 45870, 37914, 36454, 31976, 29486, 19900, 18604, 45982, 33360, 28932, 2482, 35933, 14343, 19665, 24133, 37539, 43964, 36882, 44258, 37667, 38571, 19534, 43194, 26008, 18898, 37157, 14507, 25775, 29819, 1993, 41020, 26395, 47934, 16614, 31402, 36408, 43864, 33872, 31073, 20212, 44100, 7954, 26371, 47788, 43455, 31923, 11335, 15884, 50, 20641,

Epoch 0:   1%|▏         | 5/376 [00:00<00:23, 15.85batch/s, accuracy=35.9375%, loss=1.6]

 | Epoch 0 | Loss: 1.634231686592102 | Accuracy: 17.1875%
 | Epoch 0 | Loss: 1.6047723293304443 | Accuracy: 28.125%
 | Epoch 0 | Loss: 1.6194217205047607 | Accuracy: 17.1875%
 | Epoch 0 | Loss: 1.6290189027786255 | Accuracy: 9.375%
 | Epoch 0 | Loss: 1.6191864013671875 | Accuracy: 12.5%
 | Epoch 0 | Loss: 1.604689598083496 | Accuracy: 35.9375%


Epoch 0:   3%|▎         | 11/376 [00:00<00:16, 22.79batch/s, accuracy=17.1875%, loss=1.61]

 | Epoch 0 | Loss: 1.6134064197540283 | Accuracy: 23.4375%
 | Epoch 0 | Loss: 1.6076083183288574 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.5960371494293213 | Accuracy: 26.5625%
 | Epoch 0 | Loss: 1.6143065690994263 | Accuracy: 14.0625%
 | Epoch 0 | Loss: 1.602461814880371 | Accuracy: 29.6875%
 | Epoch 0 | Loss: 1.5972589254379272 | Accuracy: 23.4375%
 | Epoch 0 | Loss: 1.5936232805252075 | Accuracy: 23.4375%
 | Epoch 0 | Loss: 1.6110635995864868 | Accuracy: 17.1875%


...

 | Epoch 1 | Loss: 0.013907003216445446 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.0064256745390594006 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.005370334256440401 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.004187936894595623 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.003314380533993244 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.0033248637337237597 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.002866060007363558 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.1125633716583252 | Accuracy: 98.4375%
 | Epoch 1 | Loss: 0.1301446259021759 | Accuracy: 98.4375%
 | Epoch 1 | Loss: 0.0030193787533789873 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.2471773475408554 | Accuracy: 96.875%


Epoch 1: 100%|██████████| 376/376 [00:10<00:00, 37.24batch/s, accuracy=100.0%, loss=0.0022]


 | Epoch 1 | Loss: 0.0036573712714016438 | Accuracy: 100.0%
 | Epoch 1 | Loss: 0.002196420216932893 | Accuracy: 100.0%


Getting model outputs: 100%|██████████| 751/751 [00:05<00:00, 126.15batch/s]


Nominal Upsample: 153.8516129032258
Majority class size: 23847
Minority class size: 155


In [30]:
min = min1.copy()
min.extend(min2)
maj = maj1.copy()
maj.extend(maj2)
upsampled_indices = [i for i in min for x in range(100)]
upsampled_indices.extend(maj)
len(upsampled_indices)

76813

## Robust Training

In [31]:
from spuco.robust_train import CustomSampleERM

robust_model = model_factory("lenet", trainset[0][0].shape, trainset.num_classes).to(device)
rt_erm = CustomSampleERM(
    model=robust_model,
    trainset=trainset,
    batch_size=64,
    optimizer=SGD(robust_model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
    num_epochs=3,
    indices=upsampled_indices,
    device=device,
    verbose=True
)
rt_erm.train()

Epoch 0:   1%|          | 7/1201 [00:00<00:50, 23.75batch/s, accuracy=31.25%, loss=1.6]   

 | Epoch 0 | Loss: 1.6177092790603638 | Accuracy: 15.625%
 | Epoch 0 | Loss: 1.5998276472091675 | Accuracy: 23.4375%
 | Epoch 0 | Loss: 1.6223251819610596 | Accuracy: 20.3125%
 | Epoch 0 | Loss: 1.6140817403793335 | Accuracy: 14.0625%
 | Epoch 0 | Loss: 1.6153064966201782 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.6127984523773193 | Accuracy: 17.1875%
 | Epoch 0 | Loss: 1.615885615348816 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.6218453645706177 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.6292908191680908 | Accuracy: 12.5%
 | Epoch 0 | Loss: 1.6076449155807495 | Accuracy: 32.8125%
 | Epoch 0 | Loss: 1.6058567762374878 | Accuracy: 20.3125%
 | Epoch 0 | Loss: 1.601448893547058 | Accuracy: 31.25%


Epoch 0:   2%|▏         | 19/1201 [00:00<00:27, 43.33batch/s, accuracy=25.0%, loss=1.61]

 | Epoch 0 | Loss: 1.608555555343628 | Accuracy: 18.75%
 | Epoch 0 | Loss: 1.6060307025909424 | Accuracy: 31.25%
 | Epoch 0 | Loss: 1.603049874305725 | Accuracy: 21.875%
 | Epoch 0 | Loss: 1.60539972782135 | Accuracy: 39.0625%
 | Epoch 0 | Loss: 1.604050636291504 | Accuracy: 20.3125%
 | Epoch 0 | Loss: 1.6035183668136597 | Accuracy: 23.4375%
 | Epoch 0 | Loss: 1.6052852869033813 | Accuracy: 29.6875%
 | Epoch 0 | Loss: 1.6059573888778687 | Accuracy: 25.0%
 | Epoch 0 | Loss: 1.6082489490509033 | Accuracy: 23.4375%
 | Epoch 0 | Loss: 1.6043007373809814 | Accuracy: 20.3125%
 | Epoch 0 | Loss: 1.6018344163894653 | Accuracy: 28.125%
 | Epoch 0 | Loss: 1.6030176877975464 | Accuracy: 25.0%
 | Epoch 0 | Loss: 1.6050809621810913 | Accuracy: 25.0%


Epoch 0:   3%|▎         | 32/1201 [00:00<00:23, 49.16batch/s, accuracy=32.8125%, loss=1.6]

...

Epoch 2:  99%|█████████▉| 1191/1201 [00:28<00:00, 38.10batch/s, accuracy=100.0%, loss=0.000547]

 | Epoch 2 | Loss: 0.0022285704035311937 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.01012821588665247 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.011821686290204525 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.002484954195097089 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.005567620042711496 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.0005625009653158486 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.03976348042488098 | Accuracy: 98.4375%
 | Epoch 2 | Loss: 0.0005465263384394348 | Accuracy: 100.0%


Epoch 2: 100%|█████████▉| 1199/1201 [00:29<00:00, 36.74batch/s, accuracy=100.0%, loss=0.00184]

 | Epoch 2 | Loss: 0.00529113132506609 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.0016107214614748955 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.010099094361066818 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.005642475560307503 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.006198097486048937 | Accuracy: 100.0%
 | Epoch 2 | Loss: 0.14877834916114807 | Accuracy: 98.4375%
 | Epoch 2 | Loss: 0.0018436722457408905 | Accuracy: 100.0%


Epoch 2: 100%|██████████| 1201/1201 [00:29<00:00, 40.93batch/s, accuracy=100.0%, loss=0.00184]


In [32]:
from spuco.evaluate import Evaluator

rt_evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=robust_model,
    device=device,
    verbose=True
)
rt_evaluator.evaluate()

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:07,  3.34it/s]

Group (0, 0) Accuracy: 99.52718676122932


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:00<00:07,  3.16it/s]

Group (0, 1) Accuracy: 65.72104018912529


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:00<00:06,  3.15it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:08<00:00,  3.33it/s]

Group (4, 3) Accuracy: 70.45454545454545


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:08<00:00,  2.90it/s]

Group (4, 4) Accuracy: 98.98989898989899


{(0, 0): 99.52718676122932,
 (0, 1): 65.72104018912529,
 (0, 2): 76.8321513002364,
 (0, 3): 68.5579196217494,
 (0, 4): 70.92198581560284,
 (1, 0): 79.70660146699267,
 (1, 1): 99.02200488997555,
 (1, 2): 83.08823529411765,
 (1, 3): 65.19607843137256,
 (1, 4): 67.6470588235294,
 (2, 0): 76.0,
 (2, 1): 43.46666666666667,
 (2, 2): 97.86666666666666,
 (2, 3): 61.86666666666667,
 (2, 4): 43.04812834224599,
 (3, 0): 63.5678391959799,
 (3, 1): 61.20906801007557,
 (3, 2): 76.0705289672544,
 (3, 3): 99.49622166246851,
 (3, 4): 73.04785894206549,
 (4, 0): 68.01007556675063,
 (4, 1): 43.576826196473554,
 (4, 2): 44.33249370277078,
 (4, 3): 70.45454545454545,
 (4, 4): 98.98989898989899}

In [33]:
rt_evaluator.worst_group_accuracy

((2, 4), 43.04812834224599)

In [34]:
rt_evaluator.average_accuracy

98.80352254806002

In [35]:
rt_evaluator.evaluate_spurious_attribute_prediction()

20.13